<center><h1 style="color:red;" >Analysis of AirBnb Data</h1></center>

# Introduction

<p>AirBnb is one of the fastest growing companies. It provides a service where hosts with spare rooms can rent them out to guests and collects a comission. More details about the company can be found at <a href="https://www.airbnb.com/">https://www.airbnb.com/</a> </p>.
<p>Our interest is finding out what their data can tell us. What insights we can get and if this insights can help other businesses in the same industry space like travel etc.</p>

the data used has been got from <a href="https://www.kaggle.com/maxbaas/airbnb">kaggle</a>

<h3>loading the data</h3>
<p>The data is in a csv file we will be using pandas library to read the csv file.</p>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('airbnb.csv')

after loading the data we can peek at it to see what we are working with and also get a brief description

In [3]:
data.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
data.shape

(213451, 16)